In [ ]:
import pandas as pd
import jinja2

In [ ]:
dist_path = './22_12_26_Выборы_студсоветов_и_УПС_Распределение_по_участкам_База.xlsx'
districts = pd.read_excel(dist_path, sheet_name='Распределение')
voters = pd.read_excel(dist_path, sheet_name='База')
candidates = pd.read_excel('./candidates.xlsx')
candidates = candidates.merge(districts[['Подразделение', 'код']], left_on='Подразделение', right_on='Подразделение', how='left')


In [ ]:
votes = pd.read_excel('./22_12_28_Выборы_студсоветов_и_УПС_Выгрузка.xlsx', skiprows=[1,2])

In [ ]:
def check_if_voted(row, column):
    if pd.isna(row[column]):
        return pd.NA
    if row['status'] == 1:
        return row[column]
    if sum([0 if row[x] == ' ' else row[x] for x in candidates[candidates['код'] == row[column]]['id']]) > 0:
        return row[column]
    return pd.NA


for column in [f'extra_{x}' for x in ['first', 'second', 'third', 'fourth', 'fifth']]:
    votes[column] = votes.apply(lambda x: check_if_voted(x, column), axis=1)

In [ ]:
merged = voters.merge(votes, on='ac', how='left').merge(districts[['Подразделение', 'код']], left_on='first', right_on='код', how='left')

In [ ]:
merged

In [ ]:
merged_count = merged.groupby(['op', 'Подразделение', ]).count()
merged_count['Явка, %'] = merged_count['№ записи']/merged_count['ac'] * 100
merged_count = merged_count.reset_index()
op_turnout = merged_count[['op', 'Подразделение', 'Явка, %', '№ записи', 'ac']]\
            .rename({'ac': 'Всего студентов', '№ записи': 'Голосов'}, axis=1)\
            .sort_values(['Подразделение', 'Явка, %'], ascending=False)
op_turnout = op_turnout[op_turnout['Всего студентов'] > 1]
op_turnout.to_excel('Явка по ОП.xlsx', index=False)


In [ ]:
gbe = [votes.groupby(f'extra_{x}').count()['id'] for x in ['first', 'second', 'third', 'fourth', 'fifth']]
votes_by_district = pd.concat(gbe, axis=1).fillna(0).sum(axis=1)

In [ ]:
districts_with_votes = districts.merge(votes_by_district.rename('Голосов'), left_on='код', right_index=True)
districts_with_votes['Явка'] = districts_with_votes['Голосов'] / districts_with_votes['Всего'] * 100

In [ ]:
print(districts_with_votes[['Подразделение', 'Явка']].sort_values('Явка', ascending=False))

In [ ]:
with open('./template.svg') as f:
    template = jinja2.Template(f.read())

In [ ]:
districts_with_votes.index = districts_with_votes['код']

In [ ]:
turnout = {'X'+a: '-' for a in districts['код']}
turnout.update({'X'+a: f'{round(b,1)}%' for a, b in districts_with_votes['Явка'].to_dict().items()})
with open('./turnout.svg', 'w') as f:
    f.write(template.render(**turnout))

In [ ]:
!/usr/local/bin/inkscape turnout.svg -d 150 -o turnout.png